In [2]:
import numpy as np

## Matrix multipilcation

Define a function in pure Python which calculates matrix product.

In [18]:
def mat_mul(A, B: np.array):
    assert A.shape[1] == B.shape[0], "The number of cols of A must equal to the number of rows of B"
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            # YOUR CODE HERE
    return C

In [13]:
A = np.random.randint(-2, 3, size=(2, 3))
B = np.random.randint(-2, 3, size=(3, 2))
A, B

(array([[-2,  2, -1],
        [ 2, -1,  1]]),
 array([[ 1,  1],
        [ 0, -1],
        [ 1, -2]]))

In [14]:
mat_mul(A, B)

array([[-3., -2.],
       [ 3.,  1.]])

In [16]:
# Testing area
for m in range(1, 10):
    for n in range(1, 10):
        for p in range(1, 10):
            A = np.random.rand(m, n)
            B = np.random.normal(size=(n, p))
            assert np.allclose(np.dot(A, B), mat_mul(A, B))

## Why NumPy is so good?

In [19]:
n = 100
A = np.random.rand(n, n)
B = np.random.rand(n, n)

In [21]:
%%time
C = mat_mul(A, B)

CPU times: user 720 ms, sys: 8.62 ms, total: 728 ms
Wall time: 745 ms


In [22]:
%%time
D = A @ B

CPU times: user 1.51 ms, sys: 2.91 ms, total: 4.42 ms
Wall time: 5.17 ms


In [23]:
np.linalg.norm(C - D)

2.672103632970926e-13

## Hilbert matrix

This is a square matrix 

$$
\boldsymbol H_n = (a_{ij}), \quad a_{ij} = \frac 1{i+j-1}, \quad 1 \leqslant i, j \leqslant n.
$$

In [25]:
from scipy.linalg import hilbert

In [26]:
hilbert(3)

array([[1.        , 0.5       , 0.33333333],
       [0.5       , 0.33333333, 0.25      ],
       [0.33333333, 0.25      , 0.2       ]])

The Hilbert matrix is extremely ill-conditioned:

In [51]:
for n in range(1, 10):
    H_n = hilbert(n)
    eigs = np.linalg.eig(H_n).eigenvalues
    print(f"n = {n}, cond_num = {np.linalg.cond(H_n):.3e}, det = {np.linalg.det(H_n):.3e}, \
            min_eig = {eigs.min():.3e}, max_eig = {eigs.max():.3e}")

n = 1, cond_num = 1.000e+00, det = 1.000e+00,             min_eig = 1.000e+00, max_eig = 1.000e+00
n = 2, cond_num = 1.928e+01, det = 8.333e-02,             min_eig = 6.574e-02, max_eig = 1.268e+00
n = 3, cond_num = 5.241e+02, det = 4.630e-04,             min_eig = 2.687e-03, max_eig = 1.408e+00
n = 4, cond_num = 1.551e+04, det = 1.653e-07,             min_eig = 9.670e-05, max_eig = 1.500e+00
n = 5, cond_num = 4.766e+05, det = 3.749e-12,             min_eig = 3.288e-06, max_eig = 1.567e+00
n = 6, cond_num = 1.495e+07, det = 5.367e-18,             min_eig = 1.083e-07, max_eig = 1.619e+00
n = 7, cond_num = 4.754e+08, det = 4.836e-25,             min_eig = 3.494e-09, max_eig = 1.661e+00
n = 8, cond_num = 1.526e+10, det = 2.737e-33,             min_eig = 1.112e-10, max_eig = 1.696e+00
n = 9, cond_num = 4.932e+11, det = 9.720e-43,             min_eig = 3.500e-12, max_eig = 1.726e+00


Now investigate how such ill-conditoning affects the quality of solving linear systems $\boldsymbol H_n\boldsymbol x = \boldsymbol b$.

In [30]:
def solve_hilbert_system(n):
    A = hilbert(n)
    b = np.random.normal(size=n)
    x = np.linalg.solve(A, b)
    print("error norm:", np.linalg.norm(A.dot(x) - b))
    return x

In [52]:
for n in range(1, 20):
    print(f"n = {n}")
    _ = solve_hilbert_system(n)

n = 1
error norm: 0.0
n = 2
error norm: 1.5700924586837752e-16
n = 3
error norm: 1.1841390682995302e-15
n = 4
error norm: 2.3043514002227376e-13
n = 5
error norm: 4.897486132178905e-12
n = 6
error norm: 5.851558496941621e-11
n = 7
error norm: 1.197047879782e-09
n = 8
error norm: 1.7376504674055236e-07
n = 9
error norm: 1.9461406107241145e-07
n = 10
error norm: 3.036335301902161e-06
n = 11
error norm: 0.00032787070886946325
n = 12
error norm: 0.14073209776316706
n = 13
error norm: 7.353357082053073
n = 14
error norm: 1.0135864160824293
n = 15
error norm: 3.430238014301425
n = 16
error norm: 9.84894593983335
n = 17
error norm: 32.33191790369275
n = 18
error norm: 3.653441968801507
n = 19
error norm: 12.82610019277319


## Sherman—Morrison formula

Let $\boldsymbol A \in \mathbb R^{n\times n}$ be an invertible matrix and $\boldsymbol u, \boldsymbol v \in \mathbb R^n$. Prove that

$$
    (\boldsymbol A + \boldsymbol u \boldsymbol v^\top)^{-1} =
    \boldsymbol A^{-1} - \frac{\boldsymbol A^{-1} \boldsymbol u \boldsymbol v^\top \boldsymbol A^{-1}}{1 + \boldsymbol v^\top\boldsymbol A^{-1} \boldsymbol u }.
$$

If we put $\boldsymbol A = \boldsymbol I$, then

$$
    (\boldsymbol I + \boldsymbol u \boldsymbol v^\top)^{-1} = \boldsymbol I - \frac{\boldsymbol u \boldsymbol v^\top}{1 + \boldsymbol v^\top\boldsymbol u }.
$$

Check the last one experimentally:

In [38]:
def check_sherman_morrison(n, n_tests=10):
    for _ in range(n_tests):
        u = np.random.randn(n)
        v = np.random.randn(n)
        lhs = np.linalg.inv(np.eye(n) + np.outer(u, v)) 
        rhs = np.eye(n) - np.outer(u, v) / (1 + np.inner(u, v))
        assert np.allclose(lhs, rhs)

In [39]:
for n in range(1, 101):
    check_sherman_morrison(n)

## QR-decomposition

Any square matrix $\boldsymbol A$ can be factored as $\boldsymbol A = \boldsymbol {QR}$, where $\boldsymbol Q$ is otrhogonal, $\boldsymbol R$ is upper triangular.

In [56]:
A = np.eye(4) - np.diag([1, 1, 1], -1)
A

array([[ 1.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.],
       [ 0., -1.,  1.,  0.],
       [ 0.,  0., -1.,  1.]])

In [60]:
from scipy.linalg import qr
Q, R = qr(A)

In [61]:
Q

array([[-0.70710678, -0.40824829, -0.28867513,  0.5       ],
       [ 0.70710678, -0.40824829, -0.28867513,  0.5       ],
       [-0.        ,  0.81649658, -0.28867513,  0.5       ],
       [-0.        , -0.        ,  0.8660254 ,  0.5       ]])

In [62]:
Q.T.dot(Q)

array([[ 1.00000000e+00, -1.02766874e-16, -6.44092477e-17,
         1.34015774e-16],
       [-1.02766874e-16,  1.00000000e+00,  6.47192599e-18,
         0.00000000e+00],
       [-6.44092477e-17,  6.47192599e-18,  1.00000000e+00,
        -9.61481343e-17],
       [ 1.34015774e-16,  0.00000000e+00, -9.61481343e-17,
         1.00000000e+00]])

In [63]:
R

array([[-1.41421356,  0.70710678,  0.        ,  0.        ],
       [ 0.        , -1.22474487,  0.81649658,  0.        ],
       [ 0.        ,  0.        , -1.15470054,  0.8660254 ],
       [ 0.        ,  0.        ,  0.        ,  0.5       ]])

## Determinants of random matrices

In [66]:
for n in range(50, 500, 50):
    A = np.random.rand(n, n)
    B = np.random.randn(n, n)
    print(f"n = {n}", f"det(A) = {np.linalg.det(A)}, det(B) = {np.linalg.det(B)}")

n = 50 det(A) = 633617.4059593034, det(B) = -1.1177304606162048e+31
n = 100 det(A) = -5.092533271178905e+25, det(B) = -2.2008332651963453e+79
n = 150 det(A) = 2.3959508854623266e+50, det(B) = -7.414548414180079e+130
n = 200 det(A) = 1.4884224076785974e+79, det(B) = -5.421865869726501e+185
n = 250 det(A) = 1.376301742902998e+111, det(B) = 1.1866152201781836e+244
n = 300 det(A) = -9.698974277128945e+144, det(B) = -2.233302454426092e+307
n = 350 det(A) = -8.570052195673854e+181, det(B) = -inf
n = 400 det(A) = -6.940946583695544e+217, det(B) = inf
n = 450 det(A) = 9.547140619290411e+256, det(B) = -inf


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
